<a href="https://colab.research.google.com/github/atlantiquesun/Stock_ML/blob/main/train_monthly_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import pandas as pd
from sklearn import preprocessing
import datetime
from dateutil.relativedelta import relativedelta
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt

In [68]:
def prepare_train_data_series(last_company_index, train_start_month, train_window, sample_window, lag=1):
  '''
  input:
  start_month: the earliest month used in training
  train_start_month: the first day of the first month in this train set
  train_end_month: the first day of the last month in this train set

  return:
  X_train: (samples, timesteps, features)

  currently no time lag
  '''
  train_end_month = train_start_month + relativedelta(months=train_window-1)
  X_train = []
  y_train = []
  trade_tic = []

  companies = pd.read_csv("/content/drive/MyDrive/StockML /Data/companyInfo/companies_final.csv")

  for i in range(companies.shape[0]):
    if(i > last_company_index):break
    company = companies.at[i, 'githubUser']
    ticker = companies.at[i, 'symbol'].upper()

    raw = pd.read_csv("/content/drive/MyDrive/StockML /Data/processedData/cumulativeData/"+company+".csv")
    raw = raw.drop('Unnamed: 0', axis=1)

    #group GitHub data into months
    raw.date = pd.to_datetime(raw['date'])
    df = raw.groupby(pd.Grouper(key="date", freq="M")).sum()
    df['month_end'] = df.index
    df['month_start'] = df['month_end'] + datetime.timedelta(days=1)
    df['month_start'] = df['month_start'].shift(1)
    df[features_column] = df[features_column].shift(lag) #lag GitHub data
    df = df.dropna()
    scaler = MinMaxScaler()
    df[features_column] = scaler.fit_transform(df[features_column])

    #select the data within the current train window
    df = df.loc[(df.month_start >= train_start_month) & (df.month_start <= train_end_month)] 

    #group stock data into months then select the last datum in a month to calculate monthly return
    stock_data = pd.read_csv("/content/drive/MyDrive/StockML /Data/financialData/"+ticker+".csv")
    stock_data['Date'] = pd.to_datetime(stock_data['Date'])
    stock_data = stock_data.groupby(pd.Grouper(key="Date", freq="M")).nth([-1]) #index are month-ends
    stock_data['Date'] = stock_data.index
    stock_data['last_close'] = stock_data.Close.shift(1)
    stock_data['monthly_return'] = (stock_data.Close / stock_data.last_close)-1
    stock_data = stock_data.dropna()

    #the first (sample_window - 1) month do not need to have stock data, so we can pad stock_df
    pad_index = []
    for t in range(1, sample_window):
      pad_index.append(((list(stock_data["Date"])[0]+relativedelta(days=1)) - relativedelta(months=t))-relativedelta(days=1)) 
    stock_pad = pd.DataFrame(index=pad_index, columns=stock_data.columns)
    stock_pad['Date'] = stock_pad.index
    stock_data = pd.concat([stock_pad, stock_data])

    #select the data within the current train window
    stock_df = stock_data.loc[(stock_data.Date >= train_start_month) & (stock_data.Date < (train_end_month+relativedelta(months=1)))]

    #some months may not have stock data
    df = df.loc[df['month_end'].isin(list(stock_df['Date']))]
    #print("check if two are the same:", df.shape, stock_df.shape, list(df['month_end'])[0], list(stock_df['Date'])[0], list(df['month_end'])[-1], list(stock_df['Date'])[-1])

    df_X = df[features_column]
    df_y = stock_df['monthly_return']
    
    if(df.shape[0] < sample_window):
      continue
    else:
      trade_tic.append(ticker)
      for j in range(df.shape[0]-sample_window+1):
        X_train.append(df_X.values[j:j+sample_window, :])
        y_train.append(df_y.values[j+sample_window-1])
    
  X_train = np.asarray(X_train)
  y_train = np.asarray(y_train)
  
  return (X_train, y_train, trade_tic)


# Model

In [69]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.linear_model import Ridge

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor

from sklearn.model_selection import TimeSeriesSplit, GridSearchCV,RandomizedSearchCV

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

def train_lstm(X_train, y_train, n_features=1):
    
    # Initialising the RNN
    regressor = Sequential()
    # Adding the first LSTM layer and some Dropout regularisation
    regressor.add(LSTM(units = 80, return_sequences = True, input_shape = (X_train.shape[1], n_features)))
    regressor.add(Dropout(0.2))

    # Adding a second LSTM layer and some Dropout regularisation
    regressor.add(LSTM(units = 40, return_sequences = True))
    regressor.add(Dropout(0.2))

    # Adding a third LSTM layer and some Dropout regularisation
    regressor.add(LSTM(units = 20, return_sequences = False))
    regressor.add(Dropout(0.2))

    # Adding a fourth LSTM layer and some Dropout regularisation
    #regressor.add(LSTM(units = 20,return_sequences = False))
    #regressor.add(Dropout(0.2))

    # Adding the output layer
    regressor.add(Dense(units = 1, activation='tanh'))
    
    #scoring_method = 'neg_mean_absolute_error'
    # scoring_method = 'neg_mean_squared_error'
    #scoring_method = 'neg_mean_squared_log_error'
    # Compiling the RNN
    regressor.compile(optimizer = 'adam', loss = 'mean_absolute_error')

    # Fitting the RNN to the Training set
    regressor.fit(X_train, y_train, epochs = 4, batch_size = 64)
    return regressor



def evaluate_model(model, X_test, y_test):
    from sklearn.metrics import mean_squared_error
    #from sklearn.metrics import mean_squared_log_error

    from sklearn.metrics import mean_absolute_error
    from sklearn.metrics import explained_variance_score
    from sklearn.metrics import r2_score
    y_predict = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_predict)
    

    mse = mean_squared_error(y_test, y_predict)
    #msle = mean_squared_log_error(y_test, y_predict)

    explained_variance = explained_variance_score(y_test, y_predict)
    r2 = r2_score(y_test, y_predict)

    return mae



def append_return_table(df_predict, unique_datetime, y_trade_return, trade_tic, current_index):
    tmp_table = pd.DataFrame(columns=trade_tic)
    tmp_table = tmp_table.append(pd.Series(y_trade_return, index=trade_tic), ignore_index=True)
    df_predict.loc[unique_datetime[current_index]] = tmp_table.loc[0]


# Training

In [70]:
last_company_index = 17
start_month=pd.to_datetime("2010-01-01")
train_window = 36
test_window = 9
sample_window = 6
features_column = ['star', 'fork', 'issue', 'commit', 'issueClosed', 'pullRequest', 'pullRequestClosed', 'pullRequestMerged']

train_start_months = []
for i in range(0, 90):
  train_start_months.append(start_month+relativedelta(months=i))
print(train_start_months[-1], train_start_months[-1]+relativedelta(months=train_window-1), train_start_months[-1]+relativedelta(months=train_window))

#prepare trade month_start info
unique_datetime = [x+relativedelta(months=train_window+test_window) for x in train_start_months]
companies = pd.read_csv("/content/drive/MyDrive/StockML /Data/companyInfo/companies_final.csv")
unique_ticker = list(companies["symbol"])


df_predict_lr = pd.DataFrame(columns=unique_ticker, index=unique_datetime)
df_predict_lasso = pd.DataFrame(columns=unique_ticker, index=unique_datetime)
df_predict_ridge = pd.DataFrame(columns=unique_ticker, index=unique_datetime)
df_predict_rf = pd.DataFrame(columns=unique_ticker, index=unique_datetime)
df_predict_svm= pd.DataFrame(columns=unique_ticker, index=unique_datetime)
#df_predict_gbm = pd.DataFrame(columns=unique_ticker, index=unique_datetime)
#df_predict_ada = pd.DataFrame(columns=unique_ticker, index=unique_datetime)
df_predict_lstm = pd.DataFrame(columns=unique_ticker, index=unique_datetime)

df_predict_best = pd.DataFrame(columns=unique_ticker, index=unique_datetime)
df_best_model_name = pd.DataFrame(columns=['model_name'], index=unique_datetime)
evaluation_record = []

for i in range(len(unique_datetime)):
  #e.g. train_start_month = 2010-01-01, train_end_month=2012-12-01
  #test_start_month = 2013-01-01, test_end_month=2013-09-01
  #trade_month = 2013-10-01 (unique_datetime[i]), will use data from 2013-05-01

  test_start_month = train_start_months[i] + relativedelta(months=train_window)
  trade_data_start_month = unique_datetime[i] + relativedelta(months= (1-sample_window))

  X_train, y_train, _ = prepare_train_data_series(last_company_index, train_start_months[i], train_window, sample_window)
  print("train:", X_train.shape, y_train.shape)

  X_test, y_test, _ = prepare_train_data_series(last_company_index, test_start_month, test_window, sample_window)
  print("test:", X_test.shape, y_test.shape)

  X_trade, y_trade, trade_tic = prepare_train_data_series(last_company_index, trade_data_start_month, sample_window, sample_window)
  print("trade:", X_trade.shape, y_trade.shape)

  lstm_model = train_lstm(X_train, y_train, len(features_column))
  lstm_eval = evaluate_model(lstm_model, X_test, y_test)

  y_trade_lstm = lstm_model.predict(X_trade).flatten()
  append_return_table(df_predict_lstm, unique_datetime, y_trade_lstm, trade_tic, current_index=i)

  print("Trade month:", unique_datetime[i] )


2017-06-01 00:00:00 2020-05-01 00:00:00 2020-06-01 00:00:00
train: (188, 6, 8) (188,)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


test: (28, 6, 8) (28,)
trade: (7, 6, 8) (7,)
Epoch 1/4
3/3 [==============================] - 6s 20ms/step - loss: 0.0848
Epoch 2/4
3/3 [==============================] - 0s 16ms/step - loss: 0.0810
Epoch 3/4
3/3 [==============================] - 0s 16ms/step - loss: 0.0831
Epoch 4/4
3/3 [==============================] - 0s 16ms/step - loss: 0.0841
Trade month: 2013-10-01 00:00:00
train: (189, 6, 8) (189,)
test: (28, 6, 8) (28,)
trade: (7, 6, 8) (7,)
Epoch 1/4
3/3 [==============================] - 6s 18ms/step - loss: 0.0940
Epoch 2/4
3/3 [==============================] - 0s 17ms/step - loss: 0.0907
Epoch 3/4
3/3 [==============================] - 0s 18ms/step - loss: 0.0888
Epoch 4/4
3/3 [==============================] - 0s 17ms/step - loss: 0.0885
Trade month: 2013-11-01 00:00:00
train: (190, 6, 8) (190,)
test: (28, 6, 8) (28,)
trade: (8, 6, 8) (8,)
Epoch 1/4
3/3 [==============================] - 6s 19ms/step - loss: 0.0877
Epoch 2/4
3/3 [==============================] - 0s 17

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


train: (197, 6, 8) (197,)
test: (33, 6, 8) (33,)
trade: (9, 6, 8) (9,)
Epoch 1/4
4/4 [==============================] - 7s 17ms/step - loss: 0.0797
Epoch 2/4
4/4 [==============================] - 0s 18ms/step - loss: 0.0827
Epoch 3/4
4/4 [==============================] - 0s 14ms/step - loss: 0.0862
Epoch 4/4
4/4 [==============================] - 0s 14ms/step - loss: 0.0867
Trade month: 2014-07-01 00:00:00
train: (198, 6, 8) (198,)
test: (34, 6, 8) (34,)
trade: (9, 6, 8) (9,)
Epoch 1/4
4/4 [==============================] - 6s 15ms/step - loss: 0.0855
Epoch 2/4
4/4 [==============================] - 0s 14ms/step - loss: 0.0838
Epoch 3/4
4/4 [==============================] - 0s 14ms/step - loss: 0.0794
Epoch 4/4
4/4 [==============================] - 0s 14ms/step - loss: 0.0817
Trade month: 2014-08-01 00:00:00
train: (199, 6, 8) (199,)
test: (35, 6, 8) (35,)
trade: (9, 6, 8) (9,)
Epoch 1/4
4/4 [==============================] - 6s 15ms/step - loss: 0.0883
Epoch 2/4
4/4 [=============

...

test: (68, 6, 8) (68,)
trade: (17, 6, 8) (17,)
Epoch 1/4
6/6 [==============================] - 6s 19ms/step - loss: 0.0718
Epoch 2/4
6/6 [==============================] - 0s 17ms/step - loss: 0.0767
Epoch 3/4
6/6 [==============================] - 0s 16ms/step - loss: 0.0733
Epoch 4/4
6/6 [==============================] - 0s 17ms/step - loss: 0.0735
Trade month: 2020-01-01 00:00:00
train: (378, 6, 8) (378,)
test: (68, 6, 8) (68,)
trade: (17, 6, 8) (17,)
Epoch 1/4
6/6 [==============================] - 7s 19ms/step - loss: 0.0846
Epoch 2/4
6/6 [==============================] - 0s 17ms/step - loss: 0.0760
Epoch 3/4
6/6 [==============================] - 0s 17ms/step - loss: 0.0789
Epoch 4/4
6/6 [==============================] - 0s 18ms/step - loss: 0.0762
Trade month: 2020-02-01 00:00:00


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


train: (382, 6, 8) (382,)
test: (68, 6, 8) (68,)
trade: (17, 6, 8) (17,)
Epoch 1/4
6/6 [==============================] - 7s 17ms/step - loss: 0.0806
Epoch 2/4
6/6 [==============================] - 0s 16ms/step - loss: 0.0813
Epoch 3/4
6/6 [==============================] - 0s 17ms/step - loss: 0.0791
Epoch 4/4
6/6 [==============================] - 0s 17ms/step - loss: 0.0746
Trade month: 2020-03-01 00:00:00
train: (386, 6, 8) (386,)
test: (68, 6, 8) (68,)
trade: (17, 6, 8) (17,)
Epoch 1/4
7/7 [==============================] - 6s 16ms/step - loss: 0.0782
Epoch 2/4
7/7 [==============================] - 0s 17ms/step - loss: 0.0751
Epoch 3/4
7/7 [==============================] - 0s 15ms/step - loss: 0.0763
Epoch 4/4
7/7 [==============================] - 0s 15ms/step - loss: 0.0740
Trade month: 2020-04-01 00:00:00
train: (392, 6, 8) (392,)
test: (68, 6, 8) (68,)
trade: (17, 6, 8) (17,)
Epoch 1/4
7/7 [==============================] - 6s 16ms/step - loss: 0.0784
Epoch 2/4
7/7 [=======

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


test: (68, 6, 8) (68,)
trade: (17, 6, 8) (17,)
Epoch 1/4
7/7 [==============================] - 7s 19ms/step - loss: 0.0886
Epoch 2/4
7/7 [==============================] - 0s 18ms/step - loss: 0.0784
Epoch 3/4
7/7 [==============================] - 0s 16ms/step - loss: 0.0781
Epoch 4/4
7/7 [==============================] - 0s 15ms/step - loss: 0.0750
Trade month: 2020-11-01 00:00:00
train: (431, 6, 8) (431,)
test: (68, 6, 8) (68,)
trade: (17, 6, 8) (17,)
Epoch 1/4
7/7 [==============================] - 7s 16ms/step - loss: 0.0872
Epoch 2/4
7/7 [==============================] - 0s 16ms/step - loss: 0.0809
Epoch 3/4
7/7 [==============================] - 0s 16ms/step - loss: 0.0823
Epoch 4/4
7/7 [==============================] - 0s 17ms/step - loss: 0.0827
Trade month: 2020-12-01 00:00:00
train: (436, 6, 8) (436,)
test: (68, 6, 8) (68,)
trade: (18, 6, 8) (18,)
Epoch 1/4
7/7 [==============================] - 7s 19ms/step - loss: 0.0842
Epoch 2/4
7/7 [==============================] -


# Visualisation

In [39]:
def show_single_stock_accuracy(df_predict, unique_datetime, ticker):
  from matplotlib import pyplot
  prediction = df_predict[ticker]

  stock_data = pd.read_csv("/content/drive/MyDrive/StockML /Data/financialData/"+ticker+".csv")
  stock_data['Date'] = pd.to_datetime(stock_data['Date'])
  stock_data = stock_data.groupby(pd.Grouper(key="Date", freq="M")).nth([-1]) #index are month-ends
  stock_data['Date'] = stock_data.index
  stock_data['last_close'] = stock_data.Close.shift(1)
  stock_data['monthly_return'] = (stock_data.Close / stock_data.last_close)-1
  stock_data = stock_data.loc[(stock_data["Date"] >= unique_datetime[0]) & (stock_data["Date"] < (unique_datetime[-1]+relativedelta(months=1)))]


  actual = list(stock_data["monthly_return"])
  prediction = list(df_predict[ticker].dropna())
  df_predict.to_csv("/content/drive/MyDrive/StockML /Data/prediction_shop.csv")
  
  pred = pd.DataFrame(index=list(range(len(actual))), columns = ["monthly_return"])
  act = pd.DataFrame(index=list(range(len(actual))), columns = ["monthly_return"])

  pred["monthly_return"] = prediction
  act["monthly_return"] = actual
  pred.plot()
  plt.show()
  act.plot()
  plt.show()

In [40]:
show_single_stock_accuracy(df_predict_lstm, unique_datetime, "SHOP")

ValueError: ignored